In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import simpy
import random

In [2]:
class arrival_process:
    """
    Class to represent the arrival process:
    - symbol is the symbol of the process (M, E_k, etc.)
    - scipy.stats probility distribution of arrival times
    """
    
    def __init__(self, symbol, arrival_distribution):
        """
        Initialization
        """
        
        self.symbol = symbol
        self.arrival_distribution = arrival_distribution
        self.arrival_rate = float(self.arrival_distribution.stats(moments='m'))
    
    def arrival(self, environment, simulation):
        """
        While the simulation time does not exceed the maximum duration, generate customers
        according to the distribution of the arrival process.
        """
        
        while True:
            simulation.log_entry(environment.now, environment.in_queue, environment.in_service)
            
            # In the case of a poisson arrival process
            if self.symbol == "M":
                # Make a timestep based on the poisson process
                time = random.expovariate(self.arrival_rate)
                environment.arrivals.append(time)
                yield environment.timeout(time)

                # Create a customer
                customer_new = customer(environment.now, environment)
                
                environment.in_queue += 1
                environment.process(customer_new.move())

In [3]:
class service_process:
    """
    Class to represent the arrival process:
    - symbol is the symbol of the process (M, E_k, etc.)
    - scipy.stats probility distribution of service times
    """
    
    def __init__(self, symbol, service_distribution):
        """
        Initialization
        """
        
        self.symbol = symbol
        self.service_distribution = service_distribution
        self.mean_service_time = float(self.service_distribution.stats(moments='m'))
            
    def service(self):
        """
        Return the service time based on the service time distribution.
        """
        
        return self.service_distribution.rvs()

In [4]:
class queue:
    """
    Queueing node based on Kendall's notation, in which:
    - A is the arrival process
    - S is the service time distribution
    - c is the number of servers
    - K is the number of places in the system
    - N is the calling population
    - D is the queue discipline
    """
    
    def __init__(self, A, S, c, K = np.inf, N = np.inf, D = "FIFO"):
        """
        Initialization
        """
    
        self.A = A
        self.S = S
        self.c = c
        self.K = K
        self.N = N
        self.D = D
    
    def kendall_notation(self):
        """
        Return queue according to the kendall notation.
        """
        
        return "{}/{}/{}/{}/{}/{}".format(self.A.symbol,
                                          self.S.symbol,
                                          str(self.c),
                                          str(self.K),
                                          str(self.N),
                                          self.D)
    
    def utilization(self, per_server = False):
        """
        The queue utilazation (rho) is equal to arrival rate (lambda)
        multiplied with the mean service time (E(S)).
        
        If the utilization is larger than c the queue length will explode,
        become infinitely long.
        
        If per_server is True, the utilization per server will be returned.
        """
        
        utilization = self.A.arrival_rate * self.S.mean_service_time
        
        if per_server == True:
            return utilization / self.c
        else:
            return utilzation

In [5]:
class customer():
    """
    Generate customers based on the arrival process.
    """
    
    def __init__(self, arrival, environment):
        """
        Initialization
        """
        
        self.arrival = arrival
        self.environment = environment
        
    def move(self):    
        with self.environment.servers.request() as my_turn:
            yield my_turn
            
            self.environment.waiting_times.append(self.environment.now - self.arrival)
            
            self.environment.in_queue -= 1
            self.environment.in_service += 1
            
            service_time = self.environment.queue.S.service()
            yield self.environment.timeout(service_time)
            self.environment.service_times.append(service_time)
            
            self.environment.in_service -= 1
            self.environment.system_times.append(self.environment.now - self.arrival)

In [6]:
class simulation():
    """
    A discrete event simulation that simulates the queue.
    - queue is a queue based on the queue class
    - seed is a random seed to have retraceable simulations
    """
    
    def __init__(self, queue, priority = False, seed = 4):
        """
        Initialization
        """
        
        random.seed(seed)
        
        self.environment = simpy.Environment()
        self.environment.queue = queue
        
        self.environment.waiting_times = []
        self.environment.service_times = []
        self.environment.system_times = []
        self.environment.arrivals = []
        
        self.environment.in_queue = 0
        self.environment.in_service = 0
        self.log = {"Time": [],
                    "In queue": [],
                    "In service": [],
                    "In system": []}
        
        if priority == False:
            self.environment.servers = simpy.Resource(self.environment, capacity = queue.c)
        else:
            self.environment.servers = simpy.PriorityResource(self.environment, capacity = queue.c)
    
    def simulate(self, time):
        """
        Simulate the queue
        """
        
        self.environment.process(self.environment.queue.A.arrival(self.environment, self))
        self.environment.run(until = time)
        
    def log_entry(self, time, in_queue, in_service):
        """
        Update the log based on the current timestamp.
        """
        
        self.log["Time"].append(time)
        self.log["In queue"].append(in_queue)
        self.log["In service"].append(in_service)
        self.log["In system"].append(in_queue + in_service)
    
    def return_log(self, to_csv = False):
        """
        Return the log in the form of a pandas dataframe.
        
        If to_csv is True, a .csv file will be saved with the name "simulation_results.csv"
        """
        
        dataframe = pd.DataFrame.from_dict(self.log)
        
        if to_csv == True:
            dataframe.to_csv("simulation_results.csv")
        
        return dataframe
    
    def get_stats(self):
        print("The arrival rate is:      {:04.2f} seconds".format(np.mean(self.environment.arrivals)))
        print("The mean waiting time is: {:04.2f} seconds".format(np.mean(self.environment.waiting_times)))
        print("The mean service time is: {:04.2f} seconds".format(np.mean(self.environment.service_times)))

In [7]:
A = arrival_process("M", stats.poisson(1))
S = service_process("M", stats.expon(1))
c = 3

q = queue(A, S, c)
sim = simulation(q)

In [8]:
sim.simulate(1000000)

In [9]:
sim.get_stats()

print("\n------------------------------------------\n")
print("Compared to the statistical solution:")
print("The arrival rate is:      {:04.2f} seconds".format(sim.environment.queue.A.arrival_rate))
print("The mean service time is: {:04.2f} seconds".format(sim.environment.queue.S.mean_service_time))

The arrival rate is:      1.00 seconds
The mean waiting time is: 0.57 seconds
The mean service time is: 2.00 seconds

------------------------------------------

Compared to the statistical solution:
The arrival rate is:      1.00 seconds
The mean service time is: 2.00 seconds
